In [1]:
!pip install docplex cplex

     |████████████████████████████████| 634 kB 18.1 MB/s 
     |████████████████████████████████| 30.9 MB 1.3 MB/s 
  Created wheel for docplex: filename=docplex-2.22.213-py3-none-any.whl size=696882 sha256=ab856d8dd31b7197c0cade79bffe85f7a01b78deff42c69327fb9b736f6c0b4a
  Stored in directory: /root/.cache/pip/wheels/90/69/6b/1375c68a5b7ff94c40263b151c86f58bd72200bf0c465b5ba3
Successfully built docplex


In [2]:
#Importing cplex API
import sys
import cplex
import docplex.cp

#Importing numpy and random generator
import numpy as np
rand = np.random

#Importing pyplot
import matplotlib.pyplot as plt

#Importing docplex model
from docplex.mp.model import Model
import itertools

#Imports
from enum import Enum
from cplex.callbacks import UserCutCallback, LazyConstraintCallback

In [ ]:
class LazyCallback(LazyConstraintCallback):
    """Lazy constraint callback to enforce the capacity constraints.

    If used then the callback is invoked for every integer feasible
    solution CPLEX finds. For each location j it checks whether
    constraint

    sum(c in C) supply[c][j] <= (|C| - 1) * used[j]
    sum(e in C) y[e] <= (|C| - 1)

    is satisfied. If not then it adds the violated constraint as lazy
    constraint.
    """

    # Callback constructor. Fields 'locations', 'clients', 'used', 'supply'
    # are set externally after registering the callback.
    def __init__(self, env):
      super().__init__(env)

    def __call__(self):
      

        for j in C:
            served = sum(self.get_values(
                [self.y[e] for e in self.C]))
            if served > (len(self.C) - 1.0):
                print('Adding lazy constraint %s <= %d' %
                      (' + '.join(['y(%d)' % (x) for x in self.C]),
                       len(self.C) - 1))
                self.add(constraint=cplex.SparsePair(
                    [self.y[e] for e in self.C] - (len(self.C) - 1),
                    [1.0] * len(self.C) + [-(len(self.C) - 1)]),
                    sense='L',
                    rhs=0.0)

In [53]:
class Type(Enum):
  AD = 1
  B = 2
  P = 3


class Vertex:
  def __init__(self, tp, delta_in, delta_out):
    self.tp = tp

    # delta_in = minus
    self.delta_in = delta_in
    
    # delta_out = plus
    self.delta_out = delta_out

**Esempio 1**

In [ ]:
E = ["E1", "E2", "E3", "E5", "E6", "E7"]  # Edges
V = []  # Vertex
w = [1, 1, 1, 1, 1, 1]  # Weigths

In [ ]:
V.append(Vertex(tp=Type.AD, delta=([],["E1"])))
V.append(Vertex(tp=Type.P, delta=(["E1"],["E2"])))
V.append(Vertex(tp=Type.P, delta=(["E2"],["E3"])))
V.append(Vertex(tp=Type.P, delta=(["E3"],["E5"])))
V.append(Vertex(tp=Type.P, delta=(["E6"],["E7"])))
V.append(Vertex(tp=Type.P, delta=(["E7"],["E6"])))
V.append(Vertex(tp=Type.P, delta=(["E5"],[])))
V.append(Vertex(tp=Type.P, delta=([],[])))

**Esempio 2**

In [54]:
E = [i for i in range(0, 8)]    # Edges
V = []  # Vertex
w = [1 for i in range(0, 8)]  # Weigths

In [55]:
V.append(Vertex(tp=Type.AD, delta_in=[], delta_out=[0]))
V.append(Vertex(tp=Type.P, delta_in=[0], delta_out=[1]))
V.append(Vertex(tp=Type.P, delta_in=[1], delta_out=[2, 3]))
V.append(Vertex(tp=Type.P, delta_in=[2], delta_out=[4]))
V.append(Vertex(tp=Type.P, delta_in=[3, 5], delta_out=[6]))
V.append(Vertex(tp=Type.P, delta_in=[6], delta_out=[5, 7]))
V.append(Vertex(tp=Type.B, delta_in=[4], delta_out=[]))
V.append(Vertex(tp=Type.B, delta_in=[7], delta_out=[]))

In [56]:
# Model
model = Model('KEP_model')

# Decision variables
y = model.binary_var_dict(E, name='y')
f_in = model.continuous_var_dict(len(V), name='f_in')
f_out = model.continuous_var_dict(len(V), name='f_out')

In [57]:
#Function to minimize
model.maximize(model.sum((w[e] * y[e]) for e in E))

In [58]:
#Constraints
model.add_constraints(model.sum(y[e] for e in V[v].delta_in) == f_in[v] for v,_ in enumerate(V))
model.add_constraints(model.sum(y[e] for e in V[v].delta_out) == f_out[v] for v,_ in enumerate(V))
model.add_constraints((f_out[v] <= f_in[v]) for v,_ in enumerate(V) if V[v].tp == Type.P)
model.add_constraints((f_in[v] <= 1) for v,_ in enumerate(V) if V[v].tp == Type.P)
model.add_constraints((f_out[v] <= 1) for v,_ in enumerate(V) if (V[v].tp == Type.AD or V[v].tp == Type.B))

[docplex.mp.LinearConstraint[](f_out_0,LE,1),
 docplex.mp.LinearConstraint[](f_out_6,LE,1),
 docplex.mp.LinearConstraint[](f_out_7,LE,1)]

In [ ]:
clb = model.register_callback(LazyCallback)

In [59]:
solution = model.solve(log_output=True)
model.solve(log_output=True)

Version identifier: 20.1.0.1 | 2021-12-07 | 9dfdf6686
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 15 rows and 8 columns.
Aggregator did 12 substitutions.
Reduced MIP has 2 rows, 4 columns, and 6 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 2 rows and 3 columns.
MIP Presolve added 1 rows and 1 columns.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.03 sec. (0.05 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.03

docplex.mp.solution.SolveSolution(obj=6,values={y_0:1,y_1:1,y_2:1,y_4:1,..

In [60]:
print(solution)

solution for: KEP_model
objective: 6
y_0=1
y_1=1
y_2=1
y_4=1
y_5=1
y_6=1
f_in_1=1.000
f_in_2=1.000
f_in_3=1.000
f_in_4=1.000
f_in_5=1.000
f_in_6=1.000
f_out_0=1.000
f_out_1=1.000
f_out_2=1.000
f_out_3=1.000
f_out_4=1.000
f_out_5=1.000



In [ ]:
print(solution.solve_details)

status  = integer optimal solution
time    = 0.0360782 s.
problem = MILP
gap     = 0%



In [61]:
solution.get_value_dict(y)

{0: 1.0, 1: 1.0, 2: 1.0, 3: 0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 0}

In [62]:
solution.get_value_dict(f_in)

{0: 0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 0}

In [63]:
solution.get_value_dict(f_out)

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 0, 7: 0}

In [64]:
def get_edges(y):
  res = []
  for i in y:
    if(y[i] != 0):
      res.append(i)
  
  return set(res)


def get_vertex(f_in, f_out, edges, V):

  V_copy = []
  chains = []
  cycles = []

  '''
  # Eliminiamo tutti i vertici non utilizzati nella soluzione
  for i,v in enumerate(V):
    if f_in[i] != 0 or f_out[i] != 0:
      V_copy.append(v)
  '''

  # Eliminiamo tutti i vertici non utilizzati nella soluzione
  for i,v in enumerate(V):
    if f_in[i] != 0 or f_out[i] != 0:
      # Calcoliamo l'intersezione tra l'insieme degli edges entranti nel nodo v
      # e l'insieme degli edges usati nella soluzione
      aux_set = edges.intersection(set(v.delta_in))
      # Aggiorniamo la lista degli edges entranti nel nodo v
      v_aux_delta_in = list(aux_set)

      # Calcoliamo l'intersezione tra l'insieme degli edges uscenti dal nodo v
      # e l'insieme degli edges usati nella soluzione
      aux_set = edges.intersection(set(v.delta_out))
      # Aggiorniamo la lista degli edges uscenti dal nodo v
      v_aux_delta_out = list(aux_set)

      # Creiamo una copia del vertice usato nella soluzione trovata
      V_copy.append(Vertex(v.tp, v_aux_delta_in, v_aux_delta_out))

  print(len(V_copy))
  i = 0
  aux = []

  # Controlliamo tutti i vertici della soluzione
  while len(V_copy) > 0:
    
    # Controlliamo se il vertice corrente ha un flusso uscente
    if len(V_copy[i].delta_out) > 0:
      # Salviamo una copia dell'edge uscente dal vertice corrente
      edge_out = V_copy[i].delta_out[0]
      # Aggiungiamo il vertice corrente ad una lista temporanea
      aux.append(V_copy[i])

      vertex_found = False
      # Cerchiamo nella lista il vertice successivo al vertice attuale
      for j,v in enumerate(V_copy):
        if j != i:
          if len(v.delta_in) > 0:

            # Controlliamo se l'edge uscente dal nodo j-esimo è uguale
            # all'edge entrante nel vertice attuale
            if v.delta_in[0] == edge_out:

              # Rimuoviamo il vertice attuale dalla lista dei vertici
              del V_copy[i]
              vertex_found = True
              # Aggiorniamo l'indice del vertice attuale
              i = V_copy.index(v)
              break
      
      # Se non è stato trovato il vertice successivo al vertice attuale,
      # questo è l'utimo vertice di un ciclo
      if not vertex_found:
        cycles.append(aux)
        aux = []
        del V_copy[i]
        i = 0

    # Se il vertice corrente non ha un flusso uscente, questo è l'ultimo vertice
    # di una catena
    else:
      # Controlliamo se il vertice attuale non è il primo della lista temporanea
      if len(aux) > 0:
        aux.append(V_copy[i])
        chains.append(aux)
        aux = []
        del V_copy[i]
        i = 0
  
  return chains, cycles

In [ ]:
def get_all_cycles(V):
  cycles = []
  aux = []
  V_copy = V

  first_vertex = V_copy[0]
  for i,v in enumerate(V_copy):
    for j in v.delta_out:
      for k,_ in enumerate(V_copy):
        if i != k:
          if v.delta_out[j] in V_copy[k].delta_in:
            if len(V_copy[k].delta_out) > 0:
              aux.append(v.delta_out[j])
              del V_copy[k].delta_in[V_copy[k].delta_in.index(v.delta_out[j])]
              del V_copy[i].delta_out[j]
            else:
              del V_copy[k]

In [65]:
edges = get_edges(solution.get_value_dict(y))
print(edges)

{0, 1, 2, 4, 5, 6}


In [66]:
chains, cycles = get_vertex(solution.get_value_dict(f_in), solution.get_value_dict(f_out), edges, V)

7


In [68]:
for c in chains[0]:
  print(str(c.tp) + " " + str(c.delta_in) + " " + str(c.delta_out))

Type.AD [] [0]
Type.P [0] [1]
Type.P [1] [2]
Type.P [2] [4]
Type.B [4] []


In [69]:
for c in cycles[0]:
  print(str(c.tp) + " " + str(c.delta_in) + " " + str(c.delta_out))

Type.P [5] [6]
Type.P [6] [5]
